In [98]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr
import requests
from bs4 import BeautifulSoup

from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

stop_words = stopwords.words('english')
recquired_length = 0.33333
total_length = 0

In [2]:
word_embeddings = {}
def load_word_embeddings():
    f = open("/media/tatan/A0A04E3AA04E16E6/word_vectors/glove.6B.100d.txt",encoding = 'utf-8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        word_embeddings[word] = coefs
    f.close()

In [4]:
load_word_embeddings()

In [5]:
html = requests.get("https://web.uvic.ca/~sdoyle/E302/Notes/SummaryNotes.html").text
soup = BeautifulSoup(html,'html.parser')

In [6]:
title = soup.find('title').text
paras = soup.findAll('p')
text = [i.text for i in paras if len(i.text)>70]
text = "".join(text)

In [104]:
sentence_tokens = sent_tokenize(text)
total_length = len(sentence_tokens)

In [11]:
def sentencevector_avg(word_vectors):
    avg_vector = []
    if len(word_vectors)!=0:
        avg_vector = np.array([sum([vector[axis] for vector in word_vectors])/len(word_vectors) for axis in range(len(word_vectors[0]))])
    else:
        avg_vector = np.zeros((300,))
    return avg_vector

In [10]:
def sentencevector_avg_weighted(word_vectors,tmp_vectors,weight):
    for i in range(len(tmp_vectors)):
        for j in range(len(tmp_vectors[0])):
            tmp_vectors[i][j] = weight*tmp_vectors[i][j]
    word_vectors+=tmp_vectors
    avg_vector = []
    if len(word_vectors)!=0:
        avg_vector = np.array([sum([vector[axis] for vector in word_vectors])/len(word_vectors) for axis in range(len(word_vectors[0]))])
    else:
        avg_vector = np.zeros((300,))
    return avg_vector

In [59]:
def remove_stopwords_and_punctuations(sen):
    tokenizer = RegexpTokenizer(r'\w+')
    sen = " ".join(tokenizer.tokenize(sen))
    new_vec = " ".join([word for word in sen.split() if word not in stop_words])
    return new_vec

In [69]:
def get_word_vectors(sentence):
    sentence = remove_stopwords_and_punctuations(sentence)
    word_vectors = []
    for word in sentence.split():
        try:
            word_vectors.append(word_embeddings[word.lower()])
        except KeyError as e:
            print(e)
    return word_vectors

In [78]:
def getSummary(title,sentence_tokens):
    summary = []
    title_vector = sentencevector_avg(get_word_vectors(title))
    for sentence_token in sentence_tokens:
        sentence_vector = []

        sentence_vector = sentencevector_avg(get_word_vectors(sentence_token))
        
        tmp = pearsonr(sentence_vector,title_vector)[0]
        summary.append((tmp,sentence_token))
        if tmp>0.5:
            pass
            #title_vector = sentencevector_avg_weighted(nlp(title),nlp(sentence_token),0.5)
            #title_vector = sentencevector_avg_weighted(nlp(sentence_token),nlp("hell"),0)
            
    return summary

In [94]:
summary = [(i,s) for i,s in enumerate(getSummary(title,sentence_tokens))]
summary.sort(key = lambda x: x[1][0], reverse = True)

'qmp'
'qmp'


In [ ]:
total_length

In [106]:
final_summary = sorted(summary[:int(recquired_length*total_length)],key = lambda x: x[0])

In [110]:
summary = ""
for i in final_summary:
    summary+=i[1][1]+"\n"
summary

"One of the most highly valued skills in any workplace that generates a lot of words—whether written or spoken—is the ability to effectively summarize that information into a more concise and readable form.\nAdministrators may deal with reports, proposals, policy documents and briefing notes that together easily reach hundreds of pages.\nYou may have to write an executive summary of a report, describe how a project is going, convey the recommendations arising from a lengthy study.\nThis summary is about 20% of the original because the original itself is short.\nWith a longer original, a summary might be 5% or less.\nThe language of the summary puts the original ideas into new words.\nThe important information usually includes controlling ideas (purpose statements and topic sentences), major findings, and conclusions or recommendations.It usually doesn't include any of the following: non-essential background information; the author's personal comments or conjectures; introductions; long

In [ ]:
for i in [1,2,3,4,5,6,7,6]*10:
    print(nlp(":")[0].vector[0])

In [ ]:
for i in range(len(tmp1)):
    if tmp1[i][0]!=tmp2[i][0]:
        print(i,sentence_tokens[i])

In [ ]:
[token.vector for token in tmp3[0] if not token.is_punct][0]

In [ ]:
[token for token in nlp(sentence_tokens[0]) if not token.is_punct]

In [ ]:
nlp("of")[0].vector